# Velocity transformation after the coordinate transformation

Here we use the Jacobian to transform the velocities of the x, and y components to velocities of the longitude and latitude components. For that, we define the function "vel_trans" which implements the matrix multiplication
$$
\begin{bmatrix}
v_{\text{lat}} \\
v_{\text{lon}}
\end{bmatrix}
= J \cdot
\begin{bmatrix}
v_x \\
v_y
\end{bmatrix}$$
Resulting Equations
$v_{\text{lat}} = \frac{\partial \text{lat}}{\partial x} \cdot v_x + \frac{\partial \text{lat}}{\partial y} \cdot v_y$
$v_{\text{lon}} = \frac{\partial \text{lon}}{\partial x} \cdot v_x + \frac{\partial \text{lon}}{\partial y} \cdot v_y$




In [2]:
# import basic packages
import sys, os
import numpy as np

#Import packages for interpolating
from scipy.interpolate import griddata

# get current directory
path = os.getcwd()

# get parent directory
parent_directory = os.path.sep.join(path.split(os.path.sep)[:-2])
utils_directory = parent_directory+"/utils"

# add utils folder to current working path in order to access the functions
sys.path.append(utils_directory)

# function which computes particle velocity
from ipynb.fs.defs.convert_meters_per_second_to_deg_per_day import m_to_deg_r

In [3]:
def velocity_transformation_compact(latitude,longitude,siu,siv,x,y,h = 1e-2):  
    """
    Transforms velocity components from a grid-based coordinate system to geographic latitude-longitude coordinates.

    Parameters:
        latitude (ndarray): 2D array of latitude values for the grid.
        longitude (ndarray): 2D array of longitude values for the grid.
        siu (ndarray): 2D array of velocity components in the x-direction (grid-based).
        siv (ndarray): 2D array of velocity components in the y-direction (grid-based).
        x (ndarray): 2D array of x-coordinates in the computational grid.
        y (ndarray): 2D array of y-coordinates in the computational grid.
        h (float, optional): Small value used for finite difference calculation. Default is 1e-2 choosen according to the spatial
                             distance between grid points which is around 20000.

    Returns:
        tuple: Transformed velocity components in geographic coordinates (vlat, vlon) in units of degrees per day.

    Notes:
        - The function uses a Jacobian matrix to transform velocity components from grid-space to geographic space.
        - Bilinear interpolation is applied to compute the transformation function
        - The latitude and longitude derivatives with respect to grid coordinates to compute the Jacobian are calculated using
          central differences.
        - The transformed velocities are converted from meters/day to degrees/day using the helper function `m_to_deg_r`.
    """


    xplush = x+h
    xminush = x-h
    yplush = y+h
    yminush = y-h

    latitude = np.asarray(latitude)
    longitude = np.asarray(longitude)

    xpluslat = griddata((x.ravel(),y.ravel()), latitude.ravel(), (xplush.ravel(),y.ravel()), method='linear', rescale=False).reshape(latitude.shape)
    xminuslat = griddata((x.ravel(),y.ravel()), latitude.ravel(), (xminush.ravel(),y.ravel()), method='linear', rescale=False).reshape(latitude.shape)
    
    # x-derivative in latitude
    print("x-derivative in latitude")
    dlatdx = (xpluslat-xminuslat)/(2*h)
    del xpluslat
    del xminuslat

    ypluslat = griddata((x.ravel(),y.ravel()), latitude.ravel(), (x.ravel(),yplush.ravel()), method='linear', rescale=False).reshape(latitude.shape)
    yminuslat = griddata((x.ravel(),y.ravel()), latitude.ravel(), (x.ravel(),yminush.ravel()), method='linear', rescale=False).reshape(latitude.shape)

    # y-derivative in latitude
    print("y-derivative in latitude")
    dlatdy = (ypluslat-yminuslat)/(2*h)
    del ypluslat
    del yminuslat

    #Velocity transformation with the Jacobian matrix
    print("Velocity transformation with the Jacobian matrix")
    vlat = np.multiply(dlatdx[:, :, np.newaxis],siu) + np.multiply(dlatdy[:, :, np.newaxis],siv)
    del dlatdx
    del dlatdy

    xpluslon = griddata((x.ravel(),y.ravel()), longitude.ravel(), (xplush.ravel(),y.ravel()), method='linear', rescale=False).reshape(longitude.shape)
    xminuslon = griddata((x.ravel(),y.ravel()), longitude.ravel(), (xminush.ravel(),y.ravel()), method='linear', rescale=False).reshape(longitude.shape)

    # x-derivative in longitude
    print("x-derivative in longitude")
    dlondx = (xpluslon-xminuslon)/(2*h)
    del xpluslon
    del xminuslon

    ypluslon = griddata((x.ravel(),y.ravel()), longitude.ravel(), (x.ravel(),yplush.ravel()), method='linear', rescale=False).reshape(longitude.shape)
    yminuslon = griddata((x.ravel(),y.ravel()), longitude.ravel(), (x.ravel(),yminush.ravel()), method='linear', rescale=False).reshape(longitude.shape)

    # y-derivative in longitude
    print("y-derivative in longitude")
    dlondy = (ypluslon-yminuslon)/(2*h)
    del ypluslon
    del yminuslon

    #Velocity transformation with the Jacobian matrix
    print("Velocity transformation with the Jacobian matrix")
    vlon = np.multiply(dlondx[:, :, np.newaxis],siu) + np.multiply(dlondy[:, :, np.newaxis],siv)
    del dlondx 
    del dlondy

    #Unit transformation from m/day to deg/day
    print("Unit transformation from m/day to deg/day")
    return m_to_deg_r(vlat, vlon, latitude)

In [4]:
def dlatdy(latitude,longitude,x,y,h = 1e-5):  
    """
    Transforms velocity components from a grid-based coordinate system to geographic latitude-longitude coordinates.

    Parameters:
        latitude (ndarray): 2D array of latitude values for the grid.
        longitude (ndarray): 2D array of longitude values for the grid.
        x (ndarray): 2D array of x-coordinates in the computational grid.
        y (ndarray): 2D array of y-coordinates in the computational grid.
        h (float, optional): Small value used for finite difference calculation. Default is 1e-2 choosen according to the spatial
                             distance between grid points which is around 20000.

    Returns:
        tuple: Transformed velocity components in geographic coordinates (vlat, vlon) in units of degrees per day.

    Notes:
        - The function uses a Jacobian matrix to transform velocity components from grid-space to geographic space.
        - Bilinear interpolation is applied to compute the transformation function
        - The latitude and longitude derivatives with respect to grid coordinates to compute the Jacobian are calculated using
          central differences.
        
    """
    
    yplush = y+h
    yminush = y-h

    latitude = np.asarray(latitude)
    longitude = np.asarray(longitude)

    ypluslat = griddata((x.ravel(),y.ravel()), latitude.ravel(), (x.ravel(),yplush.ravel()), method='linear', rescale=False).reshape(latitude.shape)
    yminuslat = griddata((x.ravel(),y.ravel()), latitude.ravel(), (x.ravel(),yminush.ravel()), method='linear', rescale=False).reshape(latitude.shape)

    # y-derivative in latitude
    print("y-derivative in latitude")
    dlatdy = (ypluslat-yminuslat)/(2*h)
    del ypluslat
    del yminuslat


    return dlatdy


In [5]:
def Jacobian(latitude,longitude,x,y,h = 1e-5):  
    """
    Transforms velocity components from a grid-based coordinate system to geographic latitude-longitude coordinates.

    Parameters:
        latitude (ndarray): 2D array of latitude values for the grid.
        longitude (ndarray): 2D array of longitude values for the grid.
        x (ndarray): 2D array of x-coordinates in the computational grid.
        y (ndarray): 2D array of y-coordinates in the computational grid.
        h (float, optional): Small value used for finite difference calculation. Default is 1e-2 choosen according to the spatial
                             distance between grid points which is around 20000.

    Returns:
        tuple: Transformed velocity components in geographic coordinates (vlat, vlon) in units of degrees per day.

    Notes:
        - The function uses a Jacobian matrix to transform velocity components from grid-space to geographic space.
        - Bilinear interpolation is applied to compute the transformation function
        - The latitude and longitude derivatives with respect to grid coordinates to compute the Jacobian are calculated using
          central differences.
        
    """


    xplush = x+h
    xminush = x-h
    yplush = y+h
    yminush = y-h

    latitude = np.asarray(latitude)
    longitude = np.asarray(longitude)

    #LATITUDE

    xpluslat = griddata((x.ravel(),y.ravel()), latitude.ravel(), (xplush.ravel(),y.ravel()), method='linear', rescale=False).reshape(latitude.shape)
    xminuslat = griddata((x.ravel(),y.ravel()), latitude.ravel(), (xminush.ravel(),y.ravel()), method='linear', rescale=False).reshape(latitude.shape)
    
    # x-derivative in latitude
    print("x-derivative in latitude")
    dlatdx = (xpluslat-xminuslat)/(2*h)
    del xpluslat
    del xminuslat

    ypluslat = griddata((x.ravel(),y.ravel()), latitude.ravel(), (x.ravel(),yplush.ravel()), method='linear', rescale=False).reshape(latitude.shape)
    yminuslat = griddata((x.ravel(),y.ravel()), latitude.ravel(), (x.ravel(),yminush.ravel()), method='linear', rescale=False).reshape(latitude.shape)

    # y-derivative in latitude
    print("y-derivative in latitude")
    dlatdy = (ypluslat-yminuslat)/(2*h)
    del ypluslat
    del yminuslat

    #LONGITUDE

    xpluslon = griddata((x.ravel(),y.ravel()), longitude.ravel(), (xplush.ravel(),y.ravel()), method='linear', rescale=False).reshape(longitude.shape)
    xminuslon = griddata((x.ravel(),y.ravel()), longitude.ravel(), (xminush.ravel(),y.ravel()), method='linear', rescale=False).reshape(longitude.shape)

    # x-derivative in longitude
    print("x-derivative in longitude")
    dlondx = (xpluslon-xminuslon)/(2*h)
    del xpluslon
    del xminuslon

    ypluslon = griddata((x.ravel(),y.ravel()), longitude.ravel(), (x.ravel(),yplush.ravel()), method='linear', rescale=False).reshape(longitude.shape)
    yminuslon = griddata((x.ravel(),y.ravel()), longitude.ravel(), (x.ravel(),yminush.ravel()), method='linear', rescale=False).reshape(longitude.shape)

    # y-derivative in longitude
    print("y-derivative in longitude")
    dlondy = (ypluslon-yminuslon)/(2*h)
    del ypluslon
    del yminuslon

    return dlatdx, dlatdy, dlondx, dlondy


In [6]:
def Jacobian_4th_order(latitude,longitude,x,y,h = 1e-2):  
    """
    Transforms velocity components from a grid-based coordinate system to geographic latitude-longitude coordinates.

    Parameters:
        latitude (ndarray): 2D array of latitude values for the grid.
        longitude (ndarray): 2D array of longitude values for the grid.
        x (ndarray): 2D array of x-coordinates in the computational grid.
        y (ndarray): 2D array of y-coordinates in the computational grid.
        h (float, optional): Small value used for finite difference calculation. Default is 1e-2 choosen according to the spatial
                             distance between grid points which is around 20000.

    Returns:
        tuple: Transformed velocity components in geographic coordinates (vlat, vlon) in units of degrees per day.

    Notes:
        - The function uses a Jacobian matrix to transform velocity components from grid-space to geographic space.
        - Bilinear interpolation is applied to compute the transformation function
        - The latitude and longitude derivatives with respect to grid coordinates to compute the Jacobian are calculated using
          central differences.
        
    """

    xplush = x+h
    xminush = x-h
    xplus2h = x+2*h
    xminus2h = x-2*h
    yplush = y+h
    yminush = y-h
    yplus2h = y+2*h
    yminus2h = y-2*h

    latitude = np.asarray(latitude)
    longitude = np.asarray(longitude)

    #LATITUDE

    xpluslat = griddata((x.ravel(),y.ravel()), latitude.ravel(), (xplush.ravel(),y.ravel()), method='linear', rescale=False).reshape(latitude.shape)
    xminuslat = griddata((x.ravel(),y.ravel()), latitude.ravel(), (xminush.ravel(),y.ravel()), method='linear', rescale=False).reshape(latitude.shape)
    xplus2lat = griddata((x.ravel(),y.ravel()), latitude.ravel(), (xplus2h.ravel(),y.ravel()), method='linear', rescale=False).reshape(latitude.shape)
    xminus2lat = griddata((x.ravel(),y.ravel()), latitude.ravel(), (xminus2h.ravel(),y.ravel()), method='linear', rescale=False).reshape(latitude.shape)

    # x-derivative in latitude
    print("x-derivative in latitude")
    dlatdx = (-xplus2lat+8*xpluslat-8*xminuslat+xminus2lat)/(12*h)
    del xpluslat
    del xminuslat
    del xplus2lat
    del xminus2lat

    ypluslat = griddata((x.ravel(),y.ravel()), latitude.ravel(), (x.ravel(),yplush.ravel()), method='linear', rescale=False).reshape(latitude.shape)
    yminuslat = griddata((x.ravel(),y.ravel()), latitude.ravel(), (x.ravel(),yminush.ravel()), method='linear', rescale=False).reshape(latitude.shape)
    yplus2lat = griddata((x.ravel(),y.ravel()), latitude.ravel(), (x.ravel(),yplus2h.ravel()), method='linear', rescale=False).reshape(latitude.shape)
    yminus2lat = griddata((x.ravel(),y.ravel()), latitude.ravel(), (x.ravel(),yminus2h.ravel()), method='linear', rescale=False).reshape(latitude.shape)

    # y-derivative in latitude
    print("y-derivative in latitude")
    dlatdy = (-yplus2lat+8*ypluslat-8*yminuslat+yminus2lat)/(12*h)
    del ypluslat
    del yminuslat
    del yplus2lat
    del yminus2lat

    #LONGITUDE

    xpluslon = griddata((x.ravel(),y.ravel()), longitude.ravel(), (xplush.ravel(),y.ravel()), method='linear', rescale=False).reshape(longitude.shape)
    xminuslon = griddata((x.ravel(),y.ravel()), longitude.ravel(), (xminush.ravel(),y.ravel()), method='linear', rescale=False).reshape(longitude.shape)
    xplus2lon = griddata((x.ravel(),y.ravel()), longitude.ravel(), (xplus2h.ravel(),y.ravel()), method='linear', rescale=False).reshape(longitude.shape)
    xminus2lon = griddata((x.ravel(),y.ravel()), longitude.ravel(), (xminus2h.ravel(),y.ravel()), method='linear', rescale=False).reshape(longitude.shape)

    # x-derivative in longitude
    print("x-derivative in longitude")
    dlondx = (-xplus2lon+8*xpluslon-8*xminuslon+xminus2lon)/(12*h)
    del xpluslon
    del xminuslon
    del xplus2lon
    del xminus2lon

    ypluslon = griddata((x.ravel(),y.ravel()), longitude.ravel(), (x.ravel(),yplush.ravel()), method='linear', rescale=False).reshape(longitude.shape)
    yminuslon = griddata((x.ravel(),y.ravel()), longitude.ravel(), (x.ravel(),yminush.ravel()), method='linear', rescale=False).reshape(longitude.shape)
    yplus2lon = griddata((x.ravel(),y.ravel()), longitude.ravel(), (x.ravel(),yplus2h.ravel()), method='linear', rescale=False).reshape(longitude.shape)
    yminus2lon = griddata((x.ravel(),y.ravel()), longitude.ravel(), (x.ravel(),yminus2h.ravel()), method='linear', rescale=False).reshape(longitude.shape)

    # y-derivative in longitude
    print("y-derivative in longitude")
    dlondy = (-yplus2lon+8*ypluslon-8*yminuslon+yminus2lon)/(12*h)
    del ypluslon
    del yminuslon
    del yplus2lon
    del yminus2lon

    return dlatdx, dlatdy, dlondx, dlondy


In [7]:
def Jacobian_dynamic_h(latitude,longitude,x,y,e1u,e2v,h = 1e-5):  
    """
    Transforms velocity components from a grid-based coordinate system to geographic latitude-longitude coordinates.

    Parameters:
        latitude (ndarray): 2D array of latitude values for the grid.
        longitude (ndarray): 2D array of longitude values for the grid.
        x (ndarray): 2D array of x-coordinates in the computational grid.
        y (ndarray): 2D array of y-coordinates in the computational grid.
        h (float, optional): Small value used for finite difference calculation. Default is 1e-2 choosen according to the spatial
                             distance between grid points which is around 20000.

    Returns:
        tuple: Transformed velocity components in geographic coordinates (vlat, vlon) in units of degrees per day.

    Notes:
        - The function uses a Jacobian matrix to transform velocity components from grid-space to geographic space.
        - Bilinear interpolation is applied to compute the transformation function
        - The latitude and longitude derivatives with respect to grid coordinates to compute the Jacobian are calculated using
          central differences.
        
    """

    hx = e1u*h/20000
    xplush = x+hx
    xminush = x-hx
    del hx

    hy = e2v*h/20000
    yplush = y+hy
    yminush = y-hy
    del hy

    latitude = np.asarray(latitude)
    longitude = np.asarray(longitude)

    #LATITUDE

    xpluslat = griddata((x.ravel(),y.ravel()), latitude.ravel(), (xplush.ravel(),y.ravel()), method='linear', rescale=False).reshape(latitude.shape)
    xminuslat = griddata((x.ravel(),y.ravel()), latitude.ravel(), (xminush.ravel(),y.ravel()), method='linear', rescale=False).reshape(latitude.shape)
    
    # x-derivative in latitude
    print("x-derivative in latitude")
    dlatdx = (xpluslat-xminuslat)/(2*h)
    del xpluslat
    del xminuslat

    ypluslat = griddata((x.ravel(),y.ravel()), latitude.ravel(), (x.ravel(),yplush.ravel()), method='linear', rescale=False).reshape(latitude.shape)
    yminuslat = griddata((x.ravel(),y.ravel()), latitude.ravel(), (x.ravel(),yminush.ravel()), method='linear', rescale=False).reshape(latitude.shape)

    # y-derivative in latitude
    print("y-derivative in latitude")
    dlatdy = (ypluslat-yminuslat)/(2*h)
    del ypluslat
    del yminuslat

    #LONGITUDE

    xpluslon = griddata((x.ravel(),y.ravel()), longitude.ravel(), (xplush.ravel(),y.ravel()), method='linear', rescale=False).reshape(longitude.shape)
    xminuslon = griddata((x.ravel(),y.ravel()), longitude.ravel(), (xminush.ravel(),y.ravel()), method='linear', rescale=False).reshape(longitude.shape)

    # x-derivative in longitude
    print("x-derivative in longitude")
    dlondx = (xpluslon-xminuslon)/(2*h)
    del xpluslon
    del xminuslon

    ypluslon = griddata((x.ravel(),y.ravel()), longitude.ravel(), (x.ravel(),yplush.ravel()), method='linear', rescale=False).reshape(longitude.shape)
    yminuslon = griddata((x.ravel(),y.ravel()), longitude.ravel(), (x.ravel(),yminush.ravel()), method='linear', rescale=False).reshape(longitude.shape)

    # y-derivative in longitude
    print("y-derivative in longitude")
    dlondy = (ypluslon-yminuslon)/(2*h)
    del ypluslon
    del yminuslon

    return dlatdx, dlatdy, dlondx, dlondy


In [8]:

def velocity_transformation(dlatdx,dlatdy,dlondx,dlondy,siu,siv):  
   
    vlon = np.multiply(dlondx[:, :, np.newaxis],siu) + np.multiply(dlondy[:, :, np.newaxis],siv)
    vlat = np.multiply(dlatdx[:, :, np.newaxis],siu) + np.multiply(dlatdy[:, :, np.newaxis],siv)
   
    #Unit transformation from m/day to deg/day
    print("Unit transformation from m/day to deg/day")
    return vlat, vlon